# Quick Start Tutorial: Reusing Features

## Learning Objectives

In this tutorial you will learn:
1. How to access catalogs of data, entities, features, and feature lists
2. How to search for features suitable for the unit of analysis
3. How to understand an existing feature
4. How to create new features from existing features
5. How to create a new feature list from existing features

## Set up the prerequisites

Learning Objectives

In this section you will:
* start your local featurebyte server
* import libraries
* learn the about catalogs
* activate a pre-built catalog

### Load the featurebyte library and connect to the local instance of featurebyte

In [1]:
# library imports
import pandas as pd
import numpy as np
import random

# load the featurebyte SDK
import featurebyte as fb

print("FeatureByte version " + fb.version)

# start the local server, then wait for it to be healthy before proceeding
fb.playground()

2023-03-27 19:49:28.927 | INFO     | featurebyte.docker.manager:start_playground:305 | Starting featurebyte service | {}


FeatureByte version 0.1.4


2023-03-27 19:49:36.750 | INFO     | featurebyte.docker.manager:start_playground:307 | Starting local spark service | {}
2023-03-27 19:49:43.583 | INFO     | featurebyte.docker.manager:start_playground:310 | Starting documentation service | {}
2023-03-27 19:49:50.449 | INFO     | featurebyte.docker.manager:start_playground:314 | Creating local spark feature store | {}
2023-03-27 19:49:50.976 | INFO     | featurebyte.docker.manager:start_playground:336 | Dataset grocery already exists, skipping import | {}
2023-03-27 19:49:50.977 | INFO     | featurebyte.docker.manager:start_playground:336 | Dataset healthcare already exists, skipping import | {}
2023-03-27 19:49:50.977 | INFO     | featurebyte.docker.manager:start_playground:336 | Dataset creditcard already exists, skipping import | {}


### Create a pre-built catalog for this tutorial, with the data, metadata, and features already set up

Note that creating a pre-built catalog is not a step you will do in real-life. This is a function specific to this quick-start tutorial to quickly skip over many of the preparatory steps and get you to a point where you can materialize features.

In a real-life project you would do data modeling, declaring the tables, entities, and the associated metadata. This would not be a frequent task, but forms the basis for best-practice feature engineering.

In [2]:
# get the functions to create a pre-built catalog
from prebuilt_catalogs import *

# create a new catalog for this tutorial
catalog_name = create_tutorial_catalog(PrebuiltCatalog.QuickStartReusingFeatures)

Cleaning up any existing tutorial catalogs
Building a quick start catalog for reusing features named [quick start reusing features 20230327:1950]
Creating new catalog
Catalog created
Registering the source tables
Registering the entities
Tagging the entities to columns in the data tables
Populating the feature store with example features
Saving Feature(s) |████████████████████████████████████████| 5/5 [100%] in 0.0s (154.79/s)                              
Loading Feature(s) |████████████████████████████████████████| 5/5 [100%] in 0.9s (5.41/s)                               


### Example: Activate an existing catalog

In [3]:
# you can activate an existing catalog
catalog = fb.Catalog.activate(catalog_name)

### Example: Load the tables and views

In [4]:
# get the tables for this workspace
grocery_customer_table = catalog.get_table("GROCERYCUSTOMER")
grocery_items_table = catalog.get_table("INVOICEITEMS")
grocery_invoice_table = catalog.get_table("GROCERYINVOICE")
grocery_product_table = catalog.get_table("GROCERYPRODUCT")

# create the views
grocery_customer_view = grocery_customer_table.get_view()
grocery_invoice_view = grocery_invoice_table.get_view()
grocery_items_view = grocery_items_table.get_view()
grocery_product_view = grocery_product_table.get_view()

## Accessing Catalogs

Learning Objectives:

In this section you will learn how to display catalogs of:
* tables
* entities
* features
* feature lists

### Example: A catalog of tables

In [5]:
# list the tables in the catalog
catalog.list_tables()

,name,type,status,entities,created_at
0,GROCERYPRODUCT,dimension_table,DRAFT,[groceryproduct],2023-03-27 11:50:25.440
1,INVOICEITEMS,item_table,DRAFT,"[groceryinvoice, groceryproduct]",2023-03-27 11:50:23.611
2,GROCERYINVOICE,event_table,DRAFT,"[groceryinvoice, grocerycustomer]",2023-03-27 11:50:21.176
3,GROCERYCUSTOMER,scd_table,DRAFT,"[grocerycustomer, frenchstate]",2023-03-27 11:50:19.968


In [6]:
# load a table
grocery_customer_table = catalog.get_table("GROCERYCUSTOMER")

# show the metadata
grocery_customer_table.info()

{
  'name': 'GROCERYCUSTOMER',
  'created_at': '2023-03-27T11:50:19.968000',
  'updated_at': '2023-03-27T11:50:26.637000',
  'status': 'DRAFT',
  'catalog_name': 'quick start reusing features 20230327:1950',
  'record_creation_timestamp_column': 'record_available_at',
  'table_details': {
    'database_name': 'spark_catalog',
    'schema_name': 'GROCERY',
    'table_name': 'GROCERYCUSTOMER'
  },
  'entities': [
    {
      'name': 'frenchstate',
      'serving_names': [
        'FRENCHSTATE'
      ],
      'catalog_name': 'quick start reusing features 20230327:1950'
    },
    {
      'name': 'grocerycustomer',
      'serving_names': [
        'GROCERYCUSTOMERGUID'
      ],
      'catalog_name': 'quick start reusing features 20230327:1950'
    }
  ],
  'semantics': [
    'scd_surrogate_key_id',
    'scd_natural_key_id'
  ],
  'column_count': 18,
  'columns_info': None,
  'natural_key_column': 'GroceryCustomerGuid',
  'effective_timestamp_column': 'ValidFrom',
  'surrogate_key_column': 

### Example: A catalog of entities

In [7]:
# list the entities in the catalog
catalog.list_entities()

,name,serving_names,created_at
0,frenchstate,[FRENCHSTATE],2023-03-27 11:50:26.265
1,groceryproduct,[GROCERYPRODUCTGUID],2023-03-27 11:50:26.041
2,groceryinvoice,[GROCERYINVOICEGUID],2023-03-27 11:50:25.777
3,grocerycustomer,[GROCERYCUSTOMERGUID],2023-03-27 11:50:25.681


In [8]:
# list the entity relationships in the catalog
catalog.list_relationships()

,id,relationship_type,primary_entity,related_entity,primary_data_source,primary_data_type,is_enabled,created_at,updated_at
0,642183039bd87982bb5c9989,child_parent,groceryinvoice,grocerycustomer,GROCERYINVOICE,event_table,True,2023-03-27 11:50:27.199,None
1,642183024fb9b91b6e68ab6a,child_parent,grocerycustomer,frenchstate,GROCERYCUSTOMER,scd_table,True,2023-03-27 11:50:26.672,None


In [9]:
# load an entity
customer_entity = catalog.get_entity("grocerycustomer")

# show the metadata
customer_entity.info()

{
  'name': 'grocerycustomer',
  'created_at': '2023-03-27T11:50:25.681000',
  'updated_at': '2023-03-27T11:50:27.172000',
  'serving_names': [
    'GROCERYCUSTOMERGUID'
  ],
  'catalog_name': 'quick start reusing features 20230327:1950'
}

### Example: A catalog of features

In [10]:
# list the features in the catalog
catalog.list_features()

,name,dtype,readiness,online_enabled,tables,primary_tables,entities,primary_entities,created_at
0,InvoiceUniqueProductGroupCount,FLOAT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-03-27 11:50:43.947
1,InvoiceDiscountAmount,FLOAT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-03-27 11:50:42.420
2,InvoiceItemCount,FLOAT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS]",[INVOICEITEMS],[groceryinvoice],[groceryinvoice],2023-03-27 11:50:41.221
3,CustomerYearOfBirth,INT,DRAFT,False,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[grocerycustomer],[grocerycustomer],2023-03-27 11:50:39.929
4,CustomerSpend_14d,FLOAT,DRAFT,False,[GROCERYINVOICE],[GROCERYINVOICE],[grocerycustomer],[grocerycustomer],2023-03-27 11:50:39.335
5,CustomerInventory_24w,OBJECT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-03-27 11:50:38.105
6,CustomerInventory_28d,OBJECT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-03-27 11:50:36.706
7,StateMeanLongitude,FLOAT,DRAFT,False,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[frenchstate],[frenchstate],2023-03-27 11:50:35.451
8,StateMeanLatitude,FLOAT,DRAFT,False,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[frenchstate],[frenchstate],2023-03-27 11:50:34.880
9,StateAvgInvoiceAmount_28d,FLOAT,DRAFT,False,"[GROCERYCUSTOMER, GROCERYINVOICE]",[GROCERYINVOICE],[frenchstate],[frenchstate],2023-03-27 11:50:34.198


In [11]:
# load a feature
state_population = catalog.get_feature("StatePopulation")

# show the metadata
state_population.info()

{
  'name': 'StatePopulation',
  'created_at': '2023-03-27T11:50:31.252000',
  'updated_at': None,
  'entities': [
    {
      'name': 'frenchstate',
      'serving_names': [
        'FRENCHSTATE'
      ],
      'catalog_name': 'quick start reusing features 20230327:1950'
    }
  ],
  'primary_entity': [
    {
      'name': 'frenchstate',
      'serving_names': [
        'FRENCHSTATE'
      ],
      'catalog_name': 'quick start reusing features 20230327:1950'
    }
  ],
  'tables': [
    {
      'name': 'GROCERYCUSTOMER',
      'status': 'DRAFT',
      'catalog_name': 'quick start reusing features 20230327:1950'
    }
  ],
  'default_version_mode': 'AUTO',
  'version_count': 1,
  'catalog_name': 'quick start reusing features 20230327:1950',
  'dtype': 'FLOAT',
  'primary_table': [
    {
      'name': 'GROCERYCUSTOMER',
      'status': 'DRAFT',
      'catalog_name': 'quick start reusing features 20230327:1950'
    }
  ],
  'default_feature_id': '642183062d087cf91f3f354d',
  'version': {

In [12]:
# show the feature lineage for the state population feature
display(state_population.definition)

'# Generated by SDK version: 0.1.4\nfrom bson import ObjectId\nfrom featurebyte import SCDTable\n\n\n# scd_table name: "GROCERYCUSTOMER"\nscd_table = SCDTable.get_by_id(ObjectId("642182fb2d087cf91f3f3543"))\nscd_view = scd_table.get_view(\n    view_mode="manual",\n    drop_column_names=["record_available_at"],\n    column_cleaning_operations=[],\n)\nfeat = scd_view.groupby(by_keys=["State"], category=None).aggregate_asat(\n    value_column=None,\n    method="count",\n    feature_name="StatePopulation",\n    offset=None,\n    backward=True,\n    skip_fill_na=True,\n)\nfeat[feat.isnull()] = 0\nfeat.name = "StatePopulation"\noutput = feat\n'

### Example: A catalog of feature lists

In [13]:
# list the feature lists in the catalog
catalog.list_feature_lists()

,name,num_features,status,deployed,readiness_frac,online_frac,tables,entities,created_at
0,StateFeatureList,5,DRAFT,False,0.0,0.0,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...",[frenchstate],2023-03-27 11:50:45.243


In [14]:
# load the feature list
state_features = catalog.get_feature_list("StateFeatureList")

# show the metadata
state_features.info()

Loading Feature(s) |████████████████████████████████████████| 5/5 [100%] in 1.1s (4.59/s)                               


{
  'name': 'StateFeatureList',
  'created_at': '2023-03-27T11:50:45.243000',
  'updated_at': None,
  'entities': [
    {
      'name': 'frenchstate',
      'serving_names': [
        'FRENCHSTATE'
      ],
      'catalog_name': 'quick start reusing features 20230327:1950'
    }
  ],
  'primary_entity': [
    {
      'name': 'frenchstate',
      'serving_names': [
        'FRENCHSTATE'
      ],
      'catalog_name': 'quick start reusing features 20230327:1950'
    }
  ],
  'tables': [
    {
      'name': 'GROCERYPRODUCT',
      'status': 'DRAFT',
      'catalog_name': 'quick start reusing features 20230327:1950'
    },
    {
      'name': 'INVOICEITEMS',
      'status': 'DRAFT',
      'catalog_name': 'quick start reusing features 20230327:1950'
    },
    {
      'name': 'GROCERYINVOICE',
      'status': 'DRAFT',
      'catalog_name': 'quick start reusing features 20230327:1950'
    },
    {
      'name': 'GROCERYCUSTOMER',
      'status': 'DRAFT',
      'catalog_name': 'quick start re

In [15]:
# list the features in the feature list
state_features.list_features()

,name,version,dtype,readiness,online_enabled,tables,primary_tables,entities,primary_entities,created_at
0,StateMeanLongitude,V230327,FLOAT,DRAFT,False,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[frenchstate],[frenchstate],2023-03-27 11:50:35.439
1,StateMeanLatitude,V230327,FLOAT,DRAFT,False,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[frenchstate],[frenchstate],2023-03-27 11:50:34.870
2,StateAvgInvoiceAmount_28d,V230327,FLOAT,DRAFT,False,"[GROCERYCUSTOMER, GROCERYINVOICE]",[GROCERYINVOICE],[frenchstate],[frenchstate],2023-03-27 11:50:34.186
3,StateInventory_28d,V230327,OBJECT,DRAFT,False,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...",[INVOICEITEMS],[frenchstate],[frenchstate],2023-03-27 11:50:32.744
4,StatePopulation,V230327,FLOAT,DRAFT,False,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[frenchstate],[frenchstate],2023-03-27 11:50:31.242


## Search for Features

Learning Objectives

In this section, you will learn:
* what a primary entity is
* how to search for suitable features

### Concept: Primary entity

<b>Feature primary entity:</b> The primary entity of a feature defines the level of analysis for that feature. 
When a feature is a result of an aggregation grouped by multiple entities, the primary entity is a tuple of those entities. For instance, if a feature quantifies the interaction between a customer entity and a merchant entity in the past, such as the sum of transaction amounts grouped by customer and merchant in the past 4 weeks, the primary entity is the tuple of customer and merchant.

When a feature is derived for features with different primary entities, the primary entity is determined by the entity relationships, and the lowest level entity is selected as the primary entity. If the underlying entities have no relationship, the primary entity becomes a tuple of those entities. For example, if a feature compares the basket of a customer with the average basket of customers in the same city, the primary entity is the customer since the customer entity is a child of the customer city entity. However, if the feature is the distance between the customer location and the merchant location, the primary entity becomes the tuple of customer and merchant since these entities do not have any child-parent relationship.

<b>Feature List primary entity:</b> The main focus of a feature list is determined by its primary entity, which typically corresponds to the primary entity of the Use Case that the feature list was created for.

If the features within the list pertain to different primary entities, the primary entity of the feature list is selected based on the entities relationships, with the lowest level entity chosen as the primary entity. In cases where there are no relationships between entities, the primary entity may become a tuple comprising those entities.
To illustrate, consider a feature list comprising features related to card, customer, and customer city. In this case, the primary entity is the card entity since it is a child of both the customer and customer city entities. However, if the feature list also contains features for merchant and merchant city, the primary entity is a tuple of card and merchant.

<b>Use Case primary entity:</b> In a Use Case, the primary entity is the object or concept that defines its problem statement. Usually, this entity is singular, but in cases such as the recommendation engine use case, it can be a tuple of entities that interact with each other.

### Case study: Predicting customer spend

Consider a use case to predict customer spend. The unit of analysis and primary entity is grocery customer. You can use features with primary entities of grocery customer or french state (because state is a parent entity of customer).

### Example: Search for suitable features

In [16]:
# get a list of all the features in the catalog
all_features = catalog.list_features()

# filter to retain only those with grocery customer or state as their primary entity
child_entity = "groceryinvoice"
suitable_features = all_features.loc[
    [child_entity not in x for x in all_features.entities.values]
]
product_entity = "groceryproduct"
suitable_features = suitable_features.loc[
    [product_entity not in x for x in suitable_features.entities.values]
]

# show the features
display(suitable_features)

,name,dtype,readiness,online_enabled,tables,primary_tables,entities,primary_entities,created_at
3,CustomerYearOfBirth,INT,DRAFT,False,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[grocerycustomer],[grocerycustomer],2023-03-27 11:50:39.929
4,CustomerSpend_14d,FLOAT,DRAFT,False,[GROCERYINVOICE],[GROCERYINVOICE],[grocerycustomer],[grocerycustomer],2023-03-27 11:50:39.335
5,CustomerInventory_24w,OBJECT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-03-27 11:50:38.105
6,CustomerInventory_28d,OBJECT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-03-27 11:50:36.706
7,StateMeanLongitude,FLOAT,DRAFT,False,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[frenchstate],[frenchstate],2023-03-27 11:50:35.451
8,StateMeanLatitude,FLOAT,DRAFT,False,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[frenchstate],[frenchstate],2023-03-27 11:50:34.880
9,StateAvgInvoiceAmount_28d,FLOAT,DRAFT,False,"[GROCERYCUSTOMER, GROCERYINVOICE]",[GROCERYINVOICE],[frenchstate],[frenchstate],2023-03-27 11:50:34.198
10,StateInventory_28d,OBJECT,DRAFT,False,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...",[INVOICEITEMS],[frenchstate],[frenchstate],2023-03-27 11:50:32.760
11,StatePopulation,FLOAT,DRAFT,False,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[frenchstate],[frenchstate],2023-03-27 11:50:31.252
12,StateName,VARCHAR,DRAFT,False,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[grocerycustomer],[grocerycustomer],2023-03-27 11:50:30.480


In [17]:
# find suitable features that use the grocery invoice items table
grocery_items_features = suitable_features.loc[
    ["INVOICEITEMS" in x for x in suitable_features.tables.values]
]

# show the features
display(grocery_items_features)

,name,dtype,readiness,online_enabled,tables,primary_tables,entities,primary_entities,created_at
5,CustomerInventory_24w,OBJECT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-03-27 11:50:38.105
6,CustomerInventory_28d,OBJECT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-03-27 11:50:36.706
10,StateInventory_28d,OBJECT,DRAFT,False,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...",[INVOICEITEMS],[frenchstate],[frenchstate],2023-03-27 11:50:32.760


## Understand an Existing feature

Learning Objectives

In this section you will learn how to:
* load a feature from the catalog
* view the metadata of a feature
* materialize feature values
* view feature lineage as a definition file

### Example: Load a feature from the catalog

In [18]:
# get the CustomerInventory_28d feature
customer_inventory_28d = catalog.get_feature("CustomerInventory_28d")

### Example: View the metadata of a feature

In [19]:
# get a list of all the features in the catalog
all_features = catalog.list_features()

# display the current feature
display(all_features.loc[all_features.name == customer_inventory_28d.name])

,name,dtype,readiness,online_enabled,tables,primary_tables,entities,primary_entities,created_at
6,CustomerInventory_28d,OBJECT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-03-27 11:50:36.706


In [20]:
# view the detailed metadata
customer_inventory_28d.info()

{
  'name': 'CustomerInventory_28d',
  'created_at': '2023-03-27T11:50:36.706000',
  'updated_at': None,
  'entities': [
    {
      'name': 'grocerycustomer',
      'serving_names': [
        'GROCERYCUSTOMERGUID'
      ],
      'catalog_name': 'quick start reusing features 20230327:1950'
    }
  ],
  'primary_entity': [
    {
      'name': 'grocerycustomer',
      'serving_names': [
        'GROCERYCUSTOMERGUID'
      ],
      'catalog_name': 'quick start reusing features 20230327:1950'
    }
  ],
  'tables': [
    {
      'name': 'GROCERYPRODUCT',
      'status': 'DRAFT',
      'catalog_name': 'quick start reusing features 20230327:1950'
    },
    {
      'name': 'INVOICEITEMS',
      'status': 'DRAFT',
      'catalog_name': 'quick start reusing features 20230327:1950'
    },
    {
      'name': 'GROCERYINVOICE',
      'status': 'DRAFT',
      'catalog_name': 'quick start reusing features 20230327:1950'
    }
  ],
  'default_version_mode': 'AUTO',
  'version_count': 1,
  'catalog_n

### Example: Materialize sample values

In [21]:
# get some invoice IDs and invoice event timestamps from Q4 2022
filter = (grocery_invoice_view["Timestamp"].dt.year == 2022) & (grocery_invoice_view["Timestamp"].dt.month >= 10)

observation_set = (
    grocery_invoice_view[filter].sample(10)[["GroceryCustomerGuid", "Timestamp"]]
    .rename({
        "Timestamp": "POINT_IN_TIME",
        "GroceryCustomerGuid": "GROCERYCUSTOMERGUID",
    }, axis=1)
)
display(observation_set)

,GROCERYCUSTOMERGUID,POINT_IN_TIME
0,30e3fbe4-3cbe-4d51-b6ca-1f990ef9773d,2022-12-17 12:12:40
1,7484ebd5-ee65-49f8-abce-8becd7af39fb,2022-12-26 18:12:37
2,a906b457-33c7-4186-a4a8-77f2ad018c2b,2022-12-04 16:13:10
3,e0453f48-5d57-4681-84b3-0f07b15ab48e,2022-11-05 17:08:56
4,e459196f-bf0a-41a1-a307-a4cfcf41fea9,2022-11-27 17:42:11
5,dc577c97-cf65-410b-89c9-dd97a3129c2e,2022-10-26 15:04:59
6,6a24aaf2-65e4-48a5-8027-1088bf53102a,2022-11-27 22:36:12
7,9fe3df44-9060-4f12-a37b-1394d3849f56,2022-10-23 11:13:46
8,091be817-ce2c-4a3b-96e0-526b9b2a33f6,2022-11-17 10:48:59
9,4df6d77c-f7f9-4718-855d-df5b3f077bdf,2022-12-27 18:14:00


In [22]:
# display the feature values
display(customer_inventory_28d.preview(observation_set))

,GROCERYCUSTOMERGUID,POINT_IN_TIME,CustomerInventory_28d
0,30e3fbe4-3cbe-4d51-b6ca-1f990ef9773d,2022-12-17 12:12:40,"{""Bières et Cidres"":1,""Chips et Tortillas"":5,""..."
1,7484ebd5-ee65-49f8-abce-8becd7af39fb,2022-12-26 18:12:37,"{""Biscuits"":1,""Bières et Cidres"":2,""Cave à Vin..."
2,a906b457-33c7-4186-a4a8-77f2ad018c2b,2022-12-04 16:13:10,"{""Biscuits apéritifs"":1,""Bières et Cidres"":1,""..."
3,e0453f48-5d57-4681-84b3-0f07b15ab48e,2022-11-05 17:08:56,"{""Colas, Thés glacés et Sodas"":3,""Crèmes et Ch..."
4,e459196f-bf0a-41a1-a307-a4cfcf41fea9,2022-11-27 17:42:11,"{""Autres Produits Laitiers"":2,""Boucherie"":1,""C..."
5,dc577c97-cf65-410b-89c9-dd97a3129c2e,2022-10-26 15:04:59,"{""Biscuits apéritifs"":2,""Bières et Cidres"":19,..."
6,6a24aaf2-65e4-48a5-8027-1088bf53102a,2022-11-27 22:36:12,"{""Biscuits apéritifs"":2,""Bières et Cidres"":1,""..."
7,9fe3df44-9060-4f12-a37b-1394d3849f56,2022-10-23 11:13:46,"{""Bières et Cidres"":2,""Café"":1,""Cave à Vins"":1..."
8,091be817-ce2c-4a3b-96e0-526b9b2a33f6,2022-11-17 10:48:59,"{""Biscuits"":2,""Bières et Cidres"":1,""Boucherie""..."
9,4df6d77c-f7f9-4718-855d-df5b3f077bdf,2022-12-27 18:14:00,"{""Biscuits apéritifs"":4,""Biscuits"":1,""Bières e..."


### Example: View the feature lineage

In [23]:
# display the feature lineage for the feature we just loaded from the feature store
print(customer_inventory_28d.definition)

# Generated by SDK version: 0.1.4
from bson import ObjectId
from featurebyte import DimensionTable
from featurebyte import ItemTable


# item_table name: "INVOICEITEMS", event_table name: "GROCERYINVOICE"
item_table = ItemTable.get_by_id(ObjectId("642182fe2d087cf91f3f3545"))
item_view = item_table.get_view(
    event_suffix=None,
    view_mode="manual",
    drop_column_names=[],
    column_cleaning_operations=[],
    event_drop_column_names=["record_available_at"],
    event_column_cleaning_operations=[],
    event_join_column_names=[
        "Timestamp",
        "GroceryInvoiceGuid",
        "GroceryCustomerGuid",
    ],
)

# dimension_table name: "GROCERYPRODUCT"
dimension_table = DimensionTable.get_by_id(ObjectId("642183002d087cf91f3f3546"))
dimension_view = dimension_table.get_view(
    view_mode="manual", drop_column_names=[], column_cleaning_operations=[]
)
item_view.join(dimension_view, on=None, how="left", rsuffix="")
grouped = item_view.groupby(
    by_keys=["GroceryCustomerGu

## Create New Features from Existing Features

You can use existing features as inputs to new features.

Learning objectives

In this section you wil learn how to:
* create a new feature from two existing features

### Example: Create a new similarity feature from two existing features

In [24]:
# get the StateInventory_28d feature
state_inventory_28d = catalog.get_feature("StateInventory_28d")

# get the CustomerInventory_28d feature
customer_inventory_28d = catalog.get_feature("CustomerInventory_28d")

# create a new feature that is the cosine similarity of the two features
customer_state_items_similarity_28d = customer_inventory_28d.cd.cosine_similarity(
    state_inventory_28d
)
customer_state_items_similarity_28d.name = "CustomerStateItemsSimilarity_28d"
customer_state_items_similarity_28d.save()

# display the feature lineage for the feature we just created
display(customer_state_items_similarity_28d.definition)

'# Generated by SDK version: 0.1.4\nfrom bson import ObjectId\nfrom featurebyte import DimensionTable\nfrom featurebyte import ItemTable\nfrom featurebyte import SCDTable\n\n\n# item_table name: "INVOICEITEMS", event_table name: "GROCERYINVOICE"\nitem_table = ItemTable.get_by_id(ObjectId("642182fe2d087cf91f3f3545"))\nitem_view = item_table.get_view(\n    event_suffix=None,\n    view_mode="manual",\n    drop_column_names=[],\n    column_cleaning_operations=[],\n    event_drop_column_names=["record_available_at"],\n    event_column_cleaning_operations=[],\n    event_join_column_names=[\n        "Timestamp",\n        "GroceryInvoiceGuid",\n        "GroceryCustomerGuid",\n    ],\n)\n\n# dimension_table name: "GROCERYPRODUCT"\ndimension_table = DimensionTable.get_by_id(ObjectId("642183002d087cf91f3f3546"))\ndimension_view = dimension_table.get_view(\n    view_mode="manual", drop_column_names=[], column_cleaning_operations=[]\n)\nitem_view.join(dimension_view, on=None, how="left", rsuffix="")\ngrouped = item_view.groupby(\n    by_keys=["GroceryCustomerGuid"], category="ProductGroup"\n).aggregate_over(\n    value_column=None,\n    method="count",\n    windows=["28d"],\n    feature_names=["CustomerInventory_28d"],\n    feature_job_setting={\n        "blind_spot": "0s",\n        "frequency": "3600s",\n        "time_modulo_frequency": "90s",\n    },\n    skip_fill_na=True,\n)\nfeat = grouped["CustomerInventory_28d"]\n\n# scd_table name: "GROCERYCUSTOMER"\nscd_table = SCDTable.get_by_id(ObjectId("642182fb2d087cf91f3f3543"))\nscd_view = scd_table.get_view(\n    view_mode="manual",\n    drop_column_names=["record_available_at"],\n    column_cleaning_operations=[],\n)\nitem_view.join(scd_view, on=None, how="left", rsuffix="")\ngrouped_1 = item_view.groupby(\n    by_keys=["State"], category="ProductGroup"\n).aggregate_over(\n    value_column=None,\n    method="count",\n    windows=["28d"],\n    feature_names=["StateInventory_28d"],\n    feature_job_setting={\n        "blind_spot": "0s",\n        "frequency": "3600s",\n        "time_modulo_frequency": "90s",\n    },\n    skip_fill_na=True,\n)\nfeat_1 = grouped_1["StateInventory_28d"]\nfeat_2 = feat.cd.cosine_similarity(other=feat_1)\nfeat_2.name = "CustomerStateItemsSimilarity_28d"\noutput = feat_2\n'

## Create a New Feature List From Existing Features

Learning objectives

In this section you will learn how to:
* create a feature list with a primary entity suited to your use case

### Example: Create a customer level feature list

In [25]:
# get a list of all the features in the catalog
all_features = catalog.list_features()

# filter to retain only those with grocery customer or state as their primary entity
child_entity = "groceryinvoice"
suitable_features = all_features.loc[
    [child_entity not in x for x in all_features.entities.values]
]
product_entity = "groceryproduct"
suitable_features = suitable_features.loc[
    [product_entity not in x for x in suitable_features.entities.values]
]

# show the features
display(suitable_features)

,name,dtype,readiness,online_enabled,tables,primary_tables,entities,primary_entities,created_at
0,CustomerStateItemsSimilarity_28d,FLOAT,DRAFT,False,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...",[INVOICEITEMS],"[grocerycustomer, frenchstate]",[grocerycustomer],2023-03-27 11:51:11.188
4,CustomerYearOfBirth,INT,DRAFT,False,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[grocerycustomer],[grocerycustomer],2023-03-27 11:50:39.929
5,CustomerSpend_14d,FLOAT,DRAFT,False,[GROCERYINVOICE],[GROCERYINVOICE],[grocerycustomer],[grocerycustomer],2023-03-27 11:50:39.335
6,CustomerInventory_24w,OBJECT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-03-27 11:50:38.105
7,CustomerInventory_28d,OBJECT,DRAFT,False,"[GROCERYINVOICE, INVOICEITEMS, GROCERYPRODUCT]",[INVOICEITEMS],[grocerycustomer],[grocerycustomer],2023-03-27 11:50:36.706
8,StateMeanLongitude,FLOAT,DRAFT,False,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[frenchstate],[frenchstate],2023-03-27 11:50:35.451
9,StateMeanLatitude,FLOAT,DRAFT,False,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[frenchstate],[frenchstate],2023-03-27 11:50:34.880
10,StateAvgInvoiceAmount_28d,FLOAT,DRAFT,False,"[GROCERYCUSTOMER, GROCERYINVOICE]",[GROCERYINVOICE],[frenchstate],[frenchstate],2023-03-27 11:50:34.198
11,StateInventory_28d,OBJECT,DRAFT,False,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...",[INVOICEITEMS],[frenchstate],[frenchstate],2023-03-27 11:50:32.760
12,StatePopulation,FLOAT,DRAFT,False,[GROCERYCUSTOMER],[GROCERYCUSTOMER],[frenchstate],[frenchstate],2023-03-27 11:50:31.252


In [26]:
# create a new feature list from the 12 features we just searched for
customer_features = fb.FeatureList([
    catalog.get_feature(x)
    for x in suitable_features.name.values
], name="CustomerFeatures")
customer_features.save()

# display a sample of the feature list values
display(customer_features.preview(observation_set))

Saving Feature(s) |████████████████████████████████████████| 11/11 [100%] in 0.0s (333.13/s)                            
Loading Feature(s) |████████████████████████████████████████| 11/11 [100%] in 2.1s (5.35/s)                             


,GROCERYCUSTOMERGUID,POINT_IN_TIME,CustomerStateItemsSimilarity_28d,CustomerYearOfBirth,CustomerSpend_14d,CustomerInventory_24w,CustomerInventory_28d,StateMeanLongitude,StateMeanLatitude,StateAvgInvoiceAmount_28d,StateInventory_28d,StatePopulation,StateName
0,30e3fbe4-3cbe-4d51-b6ca-1f990ef9773d,2022-12-17 12:12:40,0.669049,1952,102.70,"{""Adoucissants et Soin du linge"":1,""Aide à la ...","{""Bières et Cidres"":1,""Chips et Tortillas"":5,""...",7.531211,48.049312,23.143898,"{""Adoucissants et Soin du linge"":2,""Aide à la ...",10,Alsace
1,7484ebd5-ee65-49f8-abce-8becd7af39fb,2022-12-26 18:12:37,0.605480,1977,124.99,"{""Aide à la Pâtisserie"":1,""Beurre"":1,""Biscuits...","{""Biscuits"":1,""Bières et Cidres"":2,""Cave à Vin...",0.726122,49.544972,16.976444,"{""Aide à la Pâtisserie"":2,""Animalerie, Soins e...",12,Haute-Normandie
2,a906b457-33c7-4186-a4a8-77f2ad018c2b,2022-12-04 16:13:10,0.753213,1947,35.32,"{""Biscuits apéritifs"":7,""Bières et Cidres"":1,""...","{""Biscuits apéritifs"":1,""Bières et Cidres"":1,""...",2.833399,50.675502,24.548324,"{""Adoucissants et Soin du linge"":4,""Aide à la ...",32,Nord-Pas-de-Calais
3,e0453f48-5d57-4681-84b3-0f07b15ab48e,2022-11-05 17:08:56,0.474846,1971,8.73,"{""Boucherie"":3,""Café"":3,""Chips et Tortillas"":1...","{""Colas, Thés glacés et Sodas"":3,""Crèmes et Ch...",-0.711901,44.519647,17.734167,"{""Aide à la Pâtisserie"":5,""Beurre"":3,""Biscuits...",19,Aquitaine
4,e459196f-bf0a-41a1-a307-a4cfcf41fea9,2022-11-27 17:42:11,0.424843,1988,36.48,"{""Autres Produits Laitiers"":3,""Autres"":1,""Bonb...","{""Autres Produits Laitiers"":2,""Boucherie"":1,""C...",2.331087,48.841199,21.928671,"{""Adoucissants et Soin du linge"":16,""Aide à la...",181,Île-de-France
5,dc577c97-cf65-410b-89c9-dd97a3129c2e,2022-10-26 15:04:59,0.280585,1956,43.53,"{""Biscuits apéritifs"":5,""Bières et Cidres"":80,...","{""Biscuits apéritifs"":2,""Bières et Cidres"":19,...",2.331234,48.841184,20.832834,"{""Adoucissants et Soin du linge"":17,""Aide à la...",180,Île-de-France
6,6a24aaf2-65e4-48a5-8027-1088bf53102a,2022-11-27 22:36:12,0.523235,1965,73.68,"{""Adoucissants et Soin du linge"":2,""Biscuits a...","{""Biscuits apéritifs"":2,""Bières et Cidres"":1,""...",7.531211,48.049312,19.836481,"{""Aide à la Pâtisserie"":5,""Autres Produits Lai...",10,Alsace
7,9fe3df44-9060-4f12-a37b-1394d3849f56,2022-10-23 11:13:46,0.460840,1950,45.28,"{""Adoucissants et Soin du linge"":1,""Biscuits a...","{""Bières et Cidres"":2,""Café"":1,""Cave à Vins"":1...",2.331234,48.841184,21.224713,"{""Adoucissants et Soin du linge"":17,""Aide à la...",180,Île-de-France
8,091be817-ce2c-4a3b-96e0-526b9b2a33f6,2022-11-17 10:48:59,0.784277,1952,106.72,"{""Aide à la Pâtisserie"":1,""Biscuits apéritifs""...","{""Biscuits"":2,""Bières et Cidres"":1,""Boucherie""...",2.331087,48.841199,22.128406,"{""Adoucissants et Soin du linge"":28,""Aide à la...",181,Île-de-France
9,4df6d77c-f7f9-4718-855d-df5b3f077bdf,2022-12-27 18:14:00,0.771087,1970,129.15,"{""Biscuits apéritifs"":6,""Biscuits"":12,""Bières ...","{""Biscuits apéritifs"":4,""Biscuits"":1,""Bières e...",4.462295,46.979665,25.099057,"{""Adoucissants et Soin du linge"":2,""Aide à la ...",10,Bourgogne


In [27]:
# list the feature lists in the catalog
catalog.list_feature_lists()

,name,num_features,status,deployed,readiness_frac,online_frac,tables,entities,created_at
0,CustomerFeatures,11,DRAFT,False,0.0,0.0,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...","[grocerycustomer, frenchstate]",2023-03-27 11:51:24.439
1,StateFeatureList,5,DRAFT,False,0.0,0.0,"[GROCERYCUSTOMER, GROCERYINVOICE, INVOICEITEMS...",[frenchstate],2023-03-27 11:50:45.243


## Next Steps

Now that you've completed the quick-start reusing features tutorial, you can put your knowledge into practice or learn more:<br>
1. Learn more about materializing features via the "Deep Dive Materializing Features" tutorial
2. Learn more about feature engineering via the "Deep Dive Feature Engineering" tutorial
3. Learn about data modeling via the "Deep Dive Data Modeling" tutorial